In [1]:
# Importar as bibliotecas

from datetime import date
import json
from pathlib import Path
import os
import sys

In [2]:
# Resolver os 'imports' do projeto 

PROJECT_ROOT = Path().resolve().parent.parent
sys.path.append(str(PROJECT_ROOT))

from utils.classes.pandas_dataframe import PandasDataframe as pd_dataframe

In [3]:
# Traz as opções de configuração do JSON

json_file = os.path.abspath('../../../options.json')

with open(json_file, 'r', encoding='utf-8') as j_file:
    json_data = json.load(j_file)

In [4]:
# Declaração dos caminhos

cdi_data_dir = os.path.abspath('../../../data')

cdi_dir = os.path.join(cdi_data_dir, 'cdi')
cdi_file = 'cdi_data.csv'
cdi_path = os.path.join(cdi_dir, cdi_file)

In [5]:
# Obtenção da planilha com os dados diários do CDI

df_cdi = pd_dataframe(cdi_path, None)
df_cdi.csv_to_df()

In [6]:
# Filtragem pela data dos dados

START_DATE = json_data['CONFIG']['START_DATE']
END_DATE = json_data.get("CONFIG", False).get("END_DATE", False) or str(date.today())
COLUMN_NAME = 'DATE'

df_cdi.query_date(START_DATE, END_DATE, COLUMN_NAME)

In [7]:
# Criar uma lista com os dados de valorização

list_cdi_daily = (df_cdi.df['CDI_TAX']).to_list()
list_day_date = (df_cdi.df['DATE']).to_list()
list_cdi_daily.pop() # Valorização começa em 0

first_cdi_valuation = 0
list_cdi_valuation = [first_cdi_valuation]
last_cdi_tax_calc = 1

for cdi_tax in list_cdi_daily:
    last_cdi_tax_calc =  (1 + (cdi_tax / 100)) * last_cdi_tax_calc
    list_cdi_valuation.append((last_cdi_tax_calc - 1) * 100)

In [8]:
# Geração de um '.csv' com a valorização do CDI

DICT_CDI_VALUATION = { 'CDI_TAX_VALUATION_PERCENT': list_cdi_valuation, 'DATE': list_day_date }

df_cdi_valuation = pd_dataframe(None, None)
df_cdi_valuation.dict_to_df(DICT_CDI_VALUATION)

cdi_valuation_archive = 'cdi_valuation.csv'
cdi_valuation_path = os.path.join(cdi_dir, cdi_valuation_archive)

df_cdi_valuation.df_to_csv(cdi_valuation_path)
